<a href="https://colab.research.google.com/github/viniciusgino06/environmental_diagnosis/blob/main/PMA_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Manipulação de imagens em Python
- Ferramenta interessante para desenvolvermos o trampo de PA
- Pode servir também para diversos trabalhos durante a graduação
- Conhecimento em linguagens de programação nunca é demais!

O COLAB é uma ferramenta muito interessante para iniciarmos. Aqui temos os principais pacotes de python instalados, além de 12 GB de RAM em GPU para rodarmos os códigos. Bora lá!

In [ ]:
#Iremos fazer a instalação de um pacote complementar (como se faz no anaconda: pip install nome-do-pacote)
!pip install geemap

Aqui iremos importar a biblioteca fundamental de manipulação de imagens em nuvem: O GOOGLE EARTH ENGINE. Para isso, digitamos o comando abaixo e autenticamos com o nosos email institucional. 

O interessante para nós usarmos essa ferramenta em conjunto com o COLAB é que temos armazenamento ilimitado pela conta institucional.

In [ ]:
import ee
#Se você nunca fez login no google earth engine, deve autenticar antes de inicializar
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=_mUW2DoE9LpY9pjnkijrb6mo-EO3tOFrgys-EKBOUBg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7DAp1NKitR_Pc5L3eY2EGhQXlNhRirLiQ7qKu3UkUbMr0oiZhH3L4

Successfully saved authorization token.


Importando mais bibliotecas e pacotes importantes:
- Pandas e Numpy: Ciência de dados
- Folium e geemap: manipulação de mapas
- time, os, requests...: referente ao sistema

In [ ]:
import pandas as pd
import numpy as np
import folium
import geemap
import time
import logging
import requests
import zipfile
import os
import math

ee.Initialize()

#### IMPORTANTE!
Aqui vem uma parte importante do nosos trabalho: ao acessarmos a plataforma https://code.earthengine.google.com/ nos deparamos com um ambiente de programação em javascript no qual podemos coletar geometrias referentes a uma região qualquer, façam o teste escolhendo SJC e fazendo um recorte utilizando a ferramenta de poligono, vocês irão identificar uma variavel sendo criada com os vertices (coordenadas) do poligono. Esse foi o principio que utilizamos para coonstruir a variavel coordenadas e geometria. Alguns objetos do Google Earth Engine (GEE) utilizados:
- ee.ImageCollection: coleção de imagens ao longo de uma série temporal
- ee.Image: conteúdos dentro de uma coleção
- ee.List: lista que pode ser utilizada para iterar sob uma coleção

Obtemos as seguintes coleções:
- Landsat 5: de 2000 a 2012 (30 metros c/ thermal)
- Landsat 8: de 2013 a 2021 (30 metros c/ thermal)
- Sentinel 2: de 2017 a 2021 (10 metros)

Glossário: Bounds = limites; Cloud cover = cobertura de nuvens

In [ ]:
#Poligono referente a região do banhado
coordenadas = "-45.92492886742651,-23.199948257720962,-45.88098355492651,-23.160970895077618"
x1,y1,x2,y2 = coordenadas.split(",")
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])
datasL5 = "2000-01-01,2013-01-01"
ini,fi = datasL5.split(",")
datasL8 = "2013-01-01,2021-04-14"
inicio,fim = datasL8.split(",")
colecaoL5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").filterBounds(geometria).filterDate(ini,fi).filterMetadata('CLOUD_COVER','less_than', 10)
colecaoL8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUD_COVER','less_than', 10)
colecaoS2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 5)
print("LANDSAT 5: "+str(colecaoL5.size().getInfo()))
print("LANDSAT 8: "+str(colecaoL8.size().getInfo()))
print("SENTINEL 2: "+str(colecaoS2.size().getInfo()))

LANDSAT 5: 90
LANDSAT 8: 77
SENTINEL 2: 66


Queremos uma imagem por ano na nossa coleção, para isso criamos a função organiza:
- organiza(coleção,data-inicio,data-final)
- retorna uma lista cotendo uma imagem por ano, que irá ser a nossa nova coleção

In [ ]:
def organiza(col,start,end):
  years = ee.List.sequence(start,end).getInfo()
  lista = ee.List([])
  for year in years:
    priano = col.filterDate(str(year)+'-01-01',str(year)+'-12-31').sort('CLOUD_COVER').first()
    if type(priano) == ee.image.Image:
      lista = lista.add(priano)

  return lista

In [ ]:
Lista_L5 = organiza(colecaoL5,2000,2011)
Lista_L8 = organiza(colecaoL8,2013,2020)
Lista_S2 = organiza(colecaoS2,2018,2020)

In [ ]:
Cole_L5 = ee.ImageCollection(Lista_L5)
Cole_L8 = ee.ImageCollection(Lista_L8)
Cole_S2 = ee.ImageCollection(Lista_S2)

Função para verificar as datas das nossas imagens

In [ ]:
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

In [ ]:
ymdList(Cole_L5)

['20000110',
 '20010909',
 '20020514',
 '20030720',
 '20040908',
 '20050515',
 '20060721',
 '20070708',
 '20080717',
 '20090830',
 '20100418',
 '20110905']

In [ ]:
ndvi = ee.ImageCollection("LANDSAT/LE07/C01/T1_ANNUAL_NDVI").filterDate("2000-01-01","2021-04-14")
ndvi.size().getInfo()

22

#### DISPOSIÇÃO EM MAPAS
Aqui iremos dispor nossas imagens em mapas, como se fosse no google earth, no canto superior direito você pode selecionar as camadas de interesse.

In [ ]:
Mapa = geemap.Map(center=[-23.18,-45.90], zoom=13)
Mapa.add_layer(ndvi.first(), {'bands': ['NDVI'],'palette': ['FFFFFF', 'F1B555', 'FCD163', '99B718', '74A901', '3E8601', '207401', '056201', '023B01',
    '012E01', '011D01', '011301']}, 'NDVI L7 2000')
Mapa.add_layer(Cole_L5.first(), {'bands': ['B3','B2','B1'],'min': 0, 'max': 66000, 'gamma': 3.5}, 'LANDSAT 5 2000') #Resolução radiométrica
Mapa.add_layer(Cole_L8.first(), {'bands': ['B4','B3','B2'],'min': 0, 'max': 66000, 'gamma': 3.5}, 'LANDSAT 8 2013')
Mapa.add_layer(Cole_S2.first(), {'bands': ['B4','B3','B2'],'min': 0, 'max': 66000, 'gamma': 4}, 'SENTINEL 2 2018')
Mapa

In [ ]:
import gdal

In [ ]:
import numpy as np
from osgeo import gdal
from osgeo import osr
import time


#----------------------------------------------------------------------
#Adaptação do exemplo disponibilizado em:
#https://mygeoblog.com/2017/10/06/from-gee-to-numpy-to-geotiff/
def save_gee_tiff(image,area,bandList,dummyValue,scaleValue,path_out):

    #image generation time
    #timedate = image.get('GENERATION_TIME').getInfo()
 
    #get the lat lon and add the bands/attributes/operations
    latlon = ee.Image.pixelLonLat().addBands( image.select(bandList) )
 
    #apply reducer to list
    latlon = latlon.reduceRegion(reducer=ee.Reducer.toList(), geometry=area, maxPixels=1e8, scale=scaleValue)
  
    #get data into different arrays
    lats = np.array((ee.Array(latlon.get('latitude')).getInfo()))
    lons = np.array((ee.Array(latlon.get('longitude')).getInfo()))
    arrayList = []
    for i in range(len(bandList)):
        data = np.array((ee.Array(latlon.get(bandList[i])).getInfo()),np.float32)
        arrayList.append(data)

    #get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
 
    #get number of columns and rows from coordinates
    ncols = len(uniqueLons)    
    nrows = len(uniqueLats)
    #number of bands in output image
    nband = len(bandList)
 
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0] 
    xs = uniqueLons[1] - uniqueLons[0]
 
    # create an array with dimensions of image
    arr = np.zeros([nband, nrows, ncols], np.float32) #-9999  #<<<descomentao para usar como dummy!?
 
    """ # fill the array with values
    counter = 0
    for y in range(nrows):       #linhas
        for x in range(ncols):   #colunas
            if (lats[counter] == uniqueLats[y]) and (lons[counter] == uniqueLons[x]) and (counter < (len(lats)-1)):
                #counter+=1
                for b in range(nband):         #bandas
                    arr[b,len(uniqueLats)-1-y,x] = arrayList[b][counter] # we start from lower left corner
                counter+=1 """
 
    #alternativa----------------------------------- (o código acima funciona na landsat)
    refLat = np.max(uniqueLats)
    refLon = np.min(uniqueLons)
    nItens = len(arrayList[0])
    #counter = 0
    for i in range(nItens):
        posLin = np.int64( np.round( (refLat - lats[i])/ys ) )
        #posCol = np.int64( np.round( (refLon - lons[i])/xs ) )
        posCol = np.int64( np.round( (lons[i] - refLon)/xs ) )
        for b in range(nband):
            arr[b,posLin,posCol] = arrayList[b][i]

    #SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
 
    #set the coordinate system
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
 
    #set driver
    driver = gdal.GetDriverByName('GTiff')
 
    #timestring = time.strftime("%Y%m%d_%H%M%S")
    outDs = driver.Create(path_out, ncols,nrows,nband,gdal.GDT_Float32)

    #projection settings
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(target.ExportToWkt())

    #building output image
    ind = 1
    for b in range(nband):
        bandArr = np.copy(arr[b,:,:])
        outBand = outDs.GetRasterBand(ind)
        outBand.WriteArray(bandArr)
        outBand.FlushCache()
        outBand.SetNoDataValue(dummyValue)
        ind += 1

    outDs = None
    del outDs, outBand

    return 'ok...'

In [ ]:
bandlist = ['B2','B3','B4','B8']
path = r'C:\\Users\\vinim\\OneDrive\\Área de Trabalho\\VINICIUS\\1_Sem_2021\\PMA'
for i in range(Cole_S2.size().getInfo()):
  imageInfo = ee.Image(Lista_S2.get(i)).getInfo()
  image = ee.Image(-99999).blend(ee.Image(Lista_S2.get(i)))
  path_out_img = path_out + imageInfo['id'].split('/')[-1] +'.tif'
  print('Processando...',imageInfo['id'])
  save_gee_tiff(image,geometria,bandlist,-99999,10,path)